<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/cp2/checkpoint2_submission1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling

In [21]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [24]:
# collocation points
Ncl = 10000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

In [25]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))
# Residual loss
def r_PINN(x,y,param):
    input_data=tf.concat([x,y],1)
    u = PINN(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    theta0 = pi/2 - param[0]
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / param[1] * a[0]**2 + b[0]**2
    D_01 = 1 / param[1] * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / param[1] * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / param[1] * a[1]**2 + b[1]**2

    return tf.sqrt((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y))  - 1/100

# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,param):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = PINN(input_data)
    r_pred   = r_PINN(xcl,ycl,param)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.pow(r_pred,2))

    # bc
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), param[2] ] ) ) ) ,2)

    #penalty over param boundaries
    mse_penalty = penalty(param[0],-np.pi/10,np.pi/10)+penalty(param[1],1,9)+penalty(param[2],-1.5,1.5)

    return mse_meas + mse_r + mse_bc + mse_penalty

# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,param):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,param)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_param = tape.gradient(loss_value,param)
    return loss_value, grads, grad_param

In [26]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM

regularization_strength = 1e-3

PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(2,),
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    tf.keras.layers.Reshape((1, 32)),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64)),

    tf.keras.layers.Dense(128, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    tf.keras.layers.Dense(64, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)
])

In [27]:
from scipy.interpolate import RBFInterpolator

def checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5):
    coordinates = np.column_stack((x, y))

    mesh_coordinates=np.column_stack((X.ravel(), Y.ravel()))

    s = [s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_aniso_1, s_value,s_value,s_value,s_value, s_aniso_2,s_value,s_value,s_value,s_value]

    rbf = RBFInterpolator(coordinates, t, neighbors=None, smoothing=s, kernel='thin_plate_spline', epsilon=None, degree=1)

    time_pred = rbf(mesh_coordinates)
    time_pred=time_pred.reshape(1501,1501)

    return time_pred

In [39]:
def checkpoint2_solution( x, y , t ):

    #train/val split
    xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(x, y, t, test_size=0.1)
    xmeas_train = tf.constant(xmeas_train.reshape(18, 1), dtype=tf.float64)
    ymeas_train = tf.constant(ymeas_train.reshape(18, 1), dtype=tf.float64)
    tmeas_train = tf.constant(tmeas_train.reshape(18, 1), dtype=tf.float64)
    xmeas_val = tf.constant(xmeas_val.reshape(2, 1), dtype=tf.float64)
    ymeas_val = tf.constant(ymeas_val.reshape(2, 1), dtype=tf.float64)
    tmeas_val = tf.constant(tmeas_val.reshape(2, 1), dtype=tf.float64)

    # y0 initial guess
    X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
    time_pred = checkpoint1_solution(xmeas, ymeas, tmeas, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5)
    y0_initial = Y[np.where(time_pred==np.min(time_pred))]

    # param initialization
    pi = tf.constant(np.pi,dtype=tf.float64)
    param = tf.Variable([[0.01], [3], [y0_initial[0]]], trainable=True,dtype=tf.float64)

    # Adam optimizer
    initial_learning_rate = 0.002
    tf_optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate,beta_1=0.99)

    patience = float('inf')
    patience_lr = 500
    min_delta = 1e-9
    best_val_loss = float('inf')
    wait = 0
    count = 0

    for iter in range(12000):

      # compute gradients using AD
      loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, param)

      # update neural network weights
      tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))

      loss_value_val, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val, param)

      best_weigths = None
      best_params = None

      # Early stopping
      if loss_value_val < best_val_loss - min_delta:
          best_val_loss = loss_value_val
          wait = 0
          count = 0
          best_weights = PINN.get_weights()
          best_params = param.numpy()
      else:
          wait += 1
          count += 1

          if count >= patience_lr:
            tf_optimizer.learning_rate = tf_optimizer.learning_rate * 0.9
            count = 0

          if wait >= patience:
              print('Early stopping at epoch', iter + 1)
              break

      # display intermediate results
      if ((iter+1) % 100 == 0):
        print('iter =  '+str(iter+1))
        #loss_value_np=loss_value.numpy()
        #print('loss = {:.4f}'.format(loss_value_np))
        tf.print('loss =' , loss_value)
        tf.print('loss_val_param =' , loss_value_val)

        print(param.numpy())

    return param[0], param[1], param[2]

In [37]:
# measurement points
ind_disp = 0
xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
theta_fiber, a_ratio, y0 = checkpoint2_solution( xmeas, ymeas , tmeas)

iter =  100
loss = [[1.7300935140234834e-05]]
loss_val_param = [[3.5013219931420777e-05]]
[[-0.0202273 ]
 [ 3.17624334]
 [ 0.67089952]]
iter =  200
loss = [[1.3225575025701779e-05]]
loss_val_param = [[1.602736172850893e-05]]
[[-0.0704666 ]
 [ 3.35306324]
 [ 0.72559667]]
iter =  300
loss = [[1.1629482953081272e-05]]
loss_val_param = [[2.6298739468318424e-05]]
[[-0.11864594]
 [ 3.52266358]
 [ 0.77508938]]
iter =  400
loss = [[6.6731665447265811e-06]]
loss_val_param = [[1.0852189934090679e-05]]
[[-0.1480649 ]
 [ 3.67970783]
 [ 0.83932603]]


In [38]:
theta_fiber, a_ratio, y0

(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-0.1480649])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([3.67970783])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.83932603])>)